In [2]:
import f3dasm
import numpy as np
from dataclasses import dataclass
from typing import List
from hydra.core.config_store import ConfigStore
import hydra

In [3]:
def find_class(query: str, module):
    return getattr(module, query)


In [4]:
@dataclass
class DesignConfig:
    bounds: List[float]
    dimensionality: int

@dataclass
class FunctionConfig:
    function_name: str
    noise: bool

@dataclass
class OptimizerConfig:
    optimizer_name: str


@dataclass
class SamplerConfig:
    sampler_name: str

@dataclass
class ExecutionConfig:
    iterations: int
    realizations: int

@dataclass
class Config:
    design: DesignConfig
    function: FunctionConfig
    sampler: SamplerConfig
    execution: ExecutionConfig

In [8]:
cs = ConfigStore.instance()
cs.store(name="f3dasm_config", node=Config)

@hydra.main(config_path=".", config_name="config")
def main(cfg: Config):
    print(cfg)

cs

In [ ]:
function_name = 'Ackley'
optimizer_name = 'CMAES'
sampler_name = 'RandomUniformSampling'
dimensionality = 2
iterations = 10
realizations = 4
bounds = np.tile([-1.0, 1.0], (dimensionality, 1))
noise = False
hyperparameters = {}

In [ ]:
function_class: f3dasm.Function = find_class(function_name, f3dasm.functions)
optimizer_class: f3dasm.Optimizer = find_class(optimizer_name, f3dasm.optimization)
sampler_class: f3dasm.SamplingInterface = find_class(sampler_name, f3dasm.sampling)

In [ ]:
design = f3dasm.make_nd_continuous_design(bounds=bounds, dimensions=dimensionality)
data = f3dasm.Data(designspace=design)

In [ ]:
function = function_class(dimensionality=dimensionality, noise=noise)
optimizer = optimizer_class(data=data, hyperparameters=hyperparameters)
sampler = sampler_class(doe=design)

In [ ]:
args = {'optimizer': optimizer,
'function': function,
'sampler': sampler,
'iterations': iterations,
'realizations': realizations}

In [ ]:
result = f3dasm.run_multiple_realizations(**args)
result